In [1]:
import numpy as np
import pandas as pd
from anndata import AnnData
import anndata
import scipy
from tqdm import tqdm
from scipy.sparse import csr_matrix
import scanpy as sc
from os.path import join
def to_adata(df):
    adata = AnnData(csr_matrix(df.values), obs=pd.DataFrame(index=df.index.values), var=pd.DataFrame(index=df.columns.values))
    return adata

In [2]:
raw_dir = "./raw/yao_2021"

In [4]:
meta = pd.read_csv(join(raw_dir, "metadata.csv"), index_col=0)
chunks = pd.read_csv(join(raw_dir, "matrix.csv"), index_col=0, chunksize=1000)
list_adata = []
for chunk in tqdm(chunks):
    list_adata.append(to_adata(chunk))
adata = anndata.concat(list_adata, axis=0)
assert set(adata.obs.index) == set(meta.index)
adata.obs = meta.reindex(adata.obs.index)
adata.write_h5ad("processed/yao_2021.raw.h5ad")

In [3]:
adata = sc.read_h5ad("processed/yao_2021.raw.h5ad")
tsne = pd.read_csv(join(raw_dir, "tsne.csv"), index_col=0)
adata.obs[["tsne_1", "tsne_2"]] = tsne.reindex(adata.obs.index)
df_hom = pd.read_csv('/n/holystore01/LABS/price_lab/Users/mjzhang/scTRS_data/gene_annotation/'
                     'mouse_human_homologs.txt', sep='\t')
adata = adata[:, adata.var.index.isin(df_hom["MOUSE_GENE_SYM"])]

/n/holystore01/LABS/price_lab/Users/khou/miniconda3/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [ ]:
sc.pp.filter_cells(adata, min_genes=0)
sc.pp.filter_genes(adata, min_cells=50)

adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highest_expr_genes(adata, n_top=20, )

adata = adata[:, adata.var.highly_variable]

sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.set_figure_params(dpi=150)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.tl.leiden(adata)

Trying to set attribute `.obs` of view, copying.


In [ ]:
raw_adata = adata.raw.to_adata()
raw_adata.var_names_make_unique()
raw_adata.write_h5ad("processed/yao_2021.processed.h5ad")

In [ ]:
df_cov = raw_adata.obs["n_genes"]
df_cov["const"] = 1.
df_cov.to_csv("processed/yao_2021.cov.tsv", sep='\t')